# Grundlegende Algorithmen und Verfahren der KI - Programmentwurf 12
Q1 2024 TIK

Gruppe:
- Simon Seifert
- Jonas Preusch
- Daniel Zängler
- Johannes Hoppe

## Aufgabe: Partitionsproblem
Gegeben ist eine Menge von $N$ Objekten mit den Größen $a_1,a_2,...,a_N \in \mathbb{N}$. Gesucht ist eine Teilmenge $P \subseteq \{1,2,...,N\}$ der Objekte, für welche $\sum{_{i \in P}}\ a_i = \sum{_{i \notin P}}\ a_i$ gilt.
Mit einem evolutionären Algorithmus soll eine solche Partition $P$ gefunden werden.

Mit einem evolutionären Algorithmus soll eine derartige Partition P gefunden werden. Für die
Erstellung einer Implementierung sind folgende Anforderungen zu beachten.
- Ausgangspunkt ist eine Folge von Größen $a_1, a_2,..., a_N \in \mathbb{N}$.
- Als genetische Repräsentation eines Lösungsvorschlags dient eine binäre Folge der Länge $N$. Dabei gilt $i \in P$ genau dann, wenn die $i$-te Stelle der Folge eine 1 ist.
- Implementieren Sie die Transformationen Selektion, Mutation und Rekombination.
- Wählen Sie $N \geq 10$ und starten Sie mit einer initialen Population von mindestens 50
Individuen.

---

## Programmentwurf:
Für die Lösung dieses Problems haben wir uns für die Pogrammiersprache Python entschieden.
Unser Code ist dabei in zwei Dateien aufgeteilt:
- [`back_end.py`](back_end.py)
- [`main.py`](main.py)

Sowie diesem Python Notebook, im Jupyter .ipynb Format, welches der Dokumentation des Entwurfs dient.

### Datei erläuterung:

`back_end.py` enthält die grundlegenden Strukturen, welche wir für einen evolutionären Algorithmus benötigen (Selektion, Mutation, Rekombination), sowie Funktionen zur bestimmung der 'Fitness' der Lösungen und zur Programm-initialisierung.

`main.py` dient zum ausführen des Programms. Hier werden die initialen Variablen gesetzt, mit welchen auch direkt die Funktionen aus `back_end.py` aufgerufen werden.

---

## Code:

#### `main()`
Beim Ausführen des Programms werden die Funktionen aus `back_end.py` importiert und die `main()` Funktion aufgerufen, vorrausgesetzt die Datei ist `"__main__"`.
Diese setzt zunächst die initialen Variablen:
- `initial_set` : Menge von $N$ Objekten $a_N \in \mathbb{N}$
- `amount_solutions` : Population an Lösungs-Individuen $P$
- `generations` : Anzahl an Generationen, für die der Algorithmus laufen soll
- `fitness_selection` : Anzahl an Lösungen einer Generation, die verändert und für die nächste verwendet werden sollen.

In [ ]:
import back_end

def main():
    initial_set = [20, 30, 15, 25, 10, 35, 40, 5, 50, 45]
    amount_solutions = 20
    generations = 50
    fitness_selection = 10

    best_solution = find_best_solution(initial_set, amount_solutions, generations, fitness_selection)

    print("Best solution so far:")
    print(best_solution)


if __name__ == "__main__":
    main()

Die Funktion `find_best_solution()` wird mit den gesetzten Variablen aufgerufen, und das Ergebnis wird `best_solution` zugewiesen.
`best_solution` wird am Ende des Programmes im Terminal ausgegeben, um das Ergebis des Alogrithmus zu sehen.

#### `find_best_solution()` 
`find_best_solution()`  besteht aus 2 Teilen.
1. `run_evolutionary_algorithm()` wird aufgerufen (wobei die gleichen Variablen einfach weiter gegeben werden), und einer Variable `result_population` zugewiesen.
2. `best_fitness_selection()` wird aufgerufen, um unter den Lösungen von `run_evolutionary_algorithm()` die mit der besten 'Fitness', d.h. die Lösung welche dem Ideal-Ergebnis am nähsten ist, ausgewählt und `best_solution` zugewiesen.

Schlussendlich wird `best_solution` zurück an die Main-Funktion geben.

In [ ]:
def find_best_solution(initial_set, amount_solutions, generations, fitness_selection):
    # Run the evolutionary algorithm
    result_population = run_evolutionary_algorithm(initial_set, amount_solutions, generations, fitness_selection)

    # Select the best solution from the final population
    best_solution = back_end.best_fitness_selection(result_population, 1)[0]

    return best_solution

#### `run_evolutionary_algorithm()`
`run_evolutionary_algorithm()` ruft selbst `back_end.initialization()` auf, um eine initiale Liste an Lösungen zu generieren.

In [ ]:
def run_evolutionary_algorithm(initial_set, amount_solutions, generations, fitness_selection):
    # Initialize the population
    population = back_end.initialization(initial_set, amount_solutions)

    # Evolve the population for the specified number of generations
    for _ in range(generations):
        # Select the best individuals from the population
        population = back_end.best_fitness_selection(population, fitness_selection)

        # Apply mutation to the selected individuals
        population = back_end.mutate(population, mutation_factor=4)

        # Perform crossover to generate the next generation
        population = back_end.random_crossover(population, amount_solutions)

    # Sort the final population by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)

    return population

#### `initialization()`
`initialization()` erstellt zufällige Lösungs-Vorschläge der Klasse `Solution`, indem die gleichnamige Methode `Solution` aufgerufen wird, bis zur Anzahl definiert in `amount_solutions`.
(Setzt damit die `population`-Variable)

In [ ]:
def initialization(initial_set, amount_solutions):
    initial_solutions = []
    for i in range(amount_solutions):
        initial_solutions.append(Solution(initial_set))
    return initial_solutions

#### `Solution`
Die Klasse besteht aus den Teilen:
- `initial_set` : Gleich der Variable, die in [`main()`](#main) gesetzt wurde
- `solution` : Zufällig generierte binäre Abfolge, durch [`generate_solutions()`](#generate_solutions)
- `total_sum` : Summe der Werte aus `initial_set`
- `fitness` : Bewertung der Lösung im Vergleich zum ideal Ergebnis, durch [`calculate_fitness`](#calculate_fitness)

Auch enthält sie einen Teil um später im Terminal als 'String' ausgegeben werden zu können.
Sowie die Funktion `calculate_fitness()`, welche den 'Fitness'-Wert der Lösung bestimmt

**TODO**: wie wird Fitness berechnet?

In [ ]:
class Solution:
    # initial Set is always bigger 10 so no division by 0 possible in  fitness
    def __init__(self, initial_set):
        self.initial_set = initial_set
        self.solution = generate_solutions(self.initial_set)
        self.total_sum = sum(initial_set)
        self.fitness = self.calculate_fitness()

    def __str__(self):
        return f"Solution: initial_set={self.initial_set}, solution={self.solution}, total_sum={self.total_sum}, fitness={self.fitness}"

    # first implementation of the fitness function
    def calculate_fitness(self):
        partition = [value for idx, value in enumerate(self.solution) if self.solution[idx] == 1]
        fitness = 1 / (abs(sum(partition) - (sum(self.initial_set) / 2)))
        return fitness

Daraufhin ist der Teil 
```python
# Initialize the population
population = back_end.initialization(initial_set, amount_solutions)
```
von [`run_evolutionary_algorithm`](#run_evolutionary_algorithm) erfüllt, womit die Funktion mit 
```python
for _ in range(generations):
        # Select the best individuals from the population
        population = back_end.best_fitness_selection(population, 10)
        
        # Apply mutation to the selected individuals
        population = back_end.mutate(population, mutation_factor=4)

        # Perform crossover to generate the next generation
        population = back_end.random_crossover(population, amount_solutions)
```
weitermacht. 
Zunächst: [`best_fitness_selection()`](#best_fitness_selection())

#### `best_fitness_selection()`
Diese Funktion ist für den Selektion in unserem Algorithmus zuständig.
`best_fitness_selection()` sortiert die Lösungen anhand ihrer 'Fitness' und gibt die Top-10 dieser zurück.
(Überschreibt die `population`-Variable)

In [ ]:
def best_fitness_selection(solutions, output_amount):
    return sorted(solutions,key=lambda x: x.fitness, reverse=True)[:output_amount]

#### `mutate()`
Diese Funktion ist für Mutation in unserem Algorithmus zuständig.
`mutate()` ändert eine zufällige Stelle der binären Abfolge (0->1 oder 1->0)

**TODO** Anstelle von `random.choice([0, 1])` könnte der Betrag von `solution[i] - 1` genommen werden, damit eine Änderung von 1->1 oder 0->0 nicht mehr möglich ist.
(Überschreibt die `population`-Variable)

In [ ]:
import random
def mutate(solutions, mutation_factor=1):
    # TODO: check if mutation from 1 to 1 should be possilbe, or to just flip values
    # chose a mutation_factor between 0 and length of the solution. we will then choose random positions of solution to
    # redraw the values,the amount based on the mutation_factor
    # we use range() plus len() to choose which positions of the solutions to change
    for solution in solutions:
        mutations = random.sample(range(len(solution.solution)), mutation_factor)
        for i in mutations:
            solution.solution[i] = random.choice([0, 1])
    return solutions

#### `random_crossover()`
Diese Funktion ist für die Rekombination in unserem Algorithmus zuständig.
`random_crossover()` wählt aus der `population` zufällig zwei 'Eltern' aus, von denen zufällige 'Gen-Paare' miteinader getauscht werden um eine neue Lösung zu generieren.
(Überschreibt die `population`-Variable)

In [ ]:
def random_crossover(parent_solutions, num_children):
    """
    Perform random crossover between a list of parent solutions.

    This method randomly selects each gene from either parent with equal probability
    to produce a specified number of child solutions.

    Args:
    - parent_solutions (list): A list of parent solutions, where each solution is represented as a list.
    - num_children (int): The number of child solutions to generate.

    Returns:
    - list: A list of child solutions resulting from the crossover operation.
    """


    children = []
    for _ in range(num_children):
        # Randomly select parents
        parent1 = random.choice(parent_solutions)
        parent2 = random.choice(parent_solutions)

        # Perform random crossover
        child=Solution(initial_set=parent1.initial_set)
        child.solution = [random.choice(gene_pair) for gene_pair in zip(parent1.solution, parent2.solution)]

        children.append(child)

    return children

#### `run_evolutionary_algorithm()` -> Ergebnis & return
([Hauptabschnitt](#run_evolutionary_algorithm))

Nach Selektion, Mutation, und Rekombination der ganzen Population, bis zur Anzahl der definierten Generationen, wird die endgültige Population noch ein mal an hand der 'Fitness' sortiert und an [`find_best_solution`](#find_best_solution) zurück gegeben:
```python
    # Sort the final population by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)

    return population
```

#### `find_best_solution()` -> Ergebnis & return

([Hauptabschnitt](#find_best_solution))

Mit der endgültigen Population, von [`run_evolutionary_algorithm()`](#run_evolutionary_algorithm---ergebnis--return) wird der Eintrag mit der besten 'Fitness' ausgewählt und an [`main()`](#main) zurück gegeben.
```python
    # Select the best solution from the final population
    best_solution = back_end.best_fitness_selection(result_population, 1)[0]

    return best_solution
```

#### `main()` -> Ergebnis

([Hauptabschnitt](#main))

`main()` gibt zuletzt das beste gefundene Ergebniss im Terminal aus, und das Programm ist beendet.
```python
    print("Best solution so far:")
    print(best_solution)
```